In [509]:
#Import libraries
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from smart_open import smart_open
# nltk.download('stopwords')  # run once
from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
from gensim import corpora
from pprint import pprint
from pyclustering.cluster.kmedians import kmedians
from pyclustering.cluster import cluster_visualizer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import FCPS_SAMPLES
from jupyterthemes import jtplot
# jtplot.style(theme='monokai', context='notebook',
#              ticks=True, grid=False)
import math
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
import operator
# jtplot.style(theme='monokai', context='notebook',
#              ticks=True, grid=False)

In [296]:
# X, y = datasets.load_iris(return_X_y=True)
# ## slim it down to 2d,
# X = X[:, 0:2]
df= pd.read_csv("final_clustering_df.csv")
X = np.asarray(df.iloc[:,[1,2]])

In [723]:
# Creating functions
def random_medians(dat, num_clusters):
    index = np.random.randint(0, (len(dat)-1), size=num_clusters)
    points = dat[index]
    return points

def euclidean_distance(dat, rand_medians):
    dist_list = []
    for i in range(len(rand_medians)):
        summation = np.sum((X-rand_medians[i])**2, axis=1)
        dist_list.append(np.sqrt(summation))
    dist_list = np.asarray(dist_list)
    return (dist_list)

def euclidean_distance_word(dat, rand_medians,word_sim):
    dist_list = []
    for i in range(len(rand_medians)):
        summation = np.sum((X-rand_medians[i])**2, axis=1)
        summation = normalize(summation.reshape(-1,1))
        summation = summation + (1-word_sim).reshape(-1,1)
#         summation = word_sim
        dist_list.append(np.sqrt(summation))
    dist_list = np.asarray(dist_list)
    return (dist_list)

def assign_obsto_clusters(dat, dist_list):
    clusters = []
    assigned_clusters = np.argmin(dist_list, axis=0)
    for i in range(len(dist_list)):
        clusters.append(dat[assigned_clusters == i])
#     clusters = np.asarray(clusters)
    return (clusters)

def centroid_cal(assigned_clusters):
    centroid = []
    for i in range(len(assigned_clusters)):
        clust= assigned_clusters[i]
        x,y = np.median(clust[:,0]) , np.median(clust[:,1])
        centroid.append((x,y))
    centroid = np.asarray(centroid)
    return (centroid)

def clust_index_find (assigned_clusters,data):
    clust_index=[]
    X = data
    for i in range(len(assigned_clusters)):
        values = np.where((X==assigned_clusters[i][:,None]).all(-1))[1]
        clust_index.append(values)
    return (clust_index)
        
def plot_clusters(assigned_clusters,centroid_old):
    color = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    for i in range(len(assigned_clusters)):
        clust = assigned_clusters[i]
        plt.scatter(clust[:,0],clust[:,1],color=RS[i])
        plt.scatter(centroid_old[i,0],centroid_old[i,1],color=color[i],
                    label=i, marker='p', s=100, edgecolor='k')
    plt.show()
    
def k_median(data,nc):
    X=data
    rand_medians = random_medians(X,num_clusters=nc)
    centroid_old = rand_medians
    dist_array = euclidean_distance(X,rand_medians)
    assigned_clusters = assign_obsto_clusters(X,dist_array)
    clust_index = clust_index_find(assigned_clusters,X)
    centroids = centroid_cal(assigned_clusters)
    i=1
    while(np.any(centroid_old != centroids)):
        centroid_old = centroids
        dist_array = euclidean_distance(X,centroid_old)
        assigned_clusters = assign_obsto_clusters(X,dist_array)
        clust_index = clust_index_find(assigned_clusters,X)
        centroids = centroid_cal(assigned_clusters)
        i=i+1        
#         print ("old=",centroid_old ,"new=",centroids)
    return (clust_index)

def similar_prod(item,no_item,data):
    user = []
    X = np.asarray(data.iloc[:,[1,2]])
    user.append(X[item])
    wordsim = word_sim_calc(data,item)
    dist_array = euclidean_distance_word(X,user,wordsim)
    dict = { i : dist_array[:,i] for i in range(0, dist_array.shape[1] ) }
    sorted_dict = sorted(dict.items(), key=operator.itemgetter(1))
    similar_prod_index = sorted_dict[0:(no_item+1)]
    return (df.iloc[np.asarray(similar_prod_index)[:,0]])

In [724]:
def word_sim_calc(df,item):
    file_docs = []
    for i in range(len(df)):
        tokens = sent_tokenize(df['Names'][i])
        file_docs.append(tokens)
#         for line in tokens:
                
#     file_docs = file_docs[0:20242]
#     gen_docs = [[w.lower() for w in word_tokenize(text)] 
#             for text in file_docs]
#     dictionary = gensim.corpora.Dictionary(gen_docs)

    mydict = corpora.Dictionary([simple_preprocess((" ".join(line))) for line in file_docs])
#     mydict = gensim.corpora.Dictionary(gen_docs)
    corpus = [mydict.doc2bow(simple_preprocess((" ".join(line)))) for line in file_docs]
    tfidf = gensim.models.TfidfModel(corpus)
#     for doc in tfidf[corpus]:
#         print([[mydict[id], np.around(freq, decimals=2)] for id, freq in doc])
    workdir = r'C:\Users\DataMining\Project\ '
    sims = gensim.similarities.Similarity(workdir,tfidf[corpus],num_features=len(dictionary))

    # perform a similarity query against the corpus
    query_doc_tf_idf = tfidf[corpus[item]]
    # print(document_number, document_similarity)
#     print('Comparing Result:', sims[query_doc_tf_idf]) 
    word_sim = sims[query_doc_tf_idf]
    return (word_sim)

In [725]:
ind = int(input("Select product index : ") )
num = int(input("Select no of similar products : ") )
similar_prod(ind,num,df)

Select product index : 66
Select no of similar products : 5


,Names,Price,Rating
66,20 Light and Dark Blue 21st Birthday Party Bal...,1.740000,4.4
10774,"Unique Party 80883 - 12"" Latex Glitz Blue 21st...",1.740000,4.4
11540,Birthday Party Decorations - Bunting Happy 21s...,1.740000,4.4
2733,"Unique Party 80919 - 12"" Latex Glitz Black and...",1.740000,4.4
640,Perfectly Pink Party Happy 70th Birthday Paper...,1.740000,4.4
7143,10 Light and Dark Blue 50th Birthday Party Bal...,21.741747,0.0


In [474]:
# index = k_median(X,5)
index

[array([    0,  2256, 10045, ..., 19806, 20233, 20237], dtype=int64),
 array([    1,  3810, 19725, ..., 20234, 20240, 20241], dtype=int64)]

In [22]:
final_clustering_df.to_csv (r'C:\Users\DataMining\Project\final_clustering_df.csv', index = None, header=True) 